 - **EntitySet**: 여러 개의 엔티티(데이터프레임)를 관리하는 객체입니다. 이를 통해 데이터 간의 관계를 정의하고, 피처 엔지니어링을 위한 기초를 마련합니다.
     - `EntitySet()`을 사용해 새로운 EntitySet을 생성.
     - `add_dataframe()`: EntitySet에 데이터프레임을 추가.
     - `normalize_dataframe()`: 데이터프레임을 정규화하여 새로운 엔티티를 생성하고, 자동으로 관계를 설정.


- **Relationships**: 서로 다른 엔티티 간의 부모-자식 관계를 정의합니다.
     - `add_relationship()`: 두 엔티티 간의 관계를 정의하고 EntitySet에 추가.

- **DFS**: 주어진 엔티티셋(EntitySet)과 관계를 기반으로 피처를 자동으로 생성하는 핵심 기능.
     - `ft.dfs()`: 자동 피처 생성 함수로, 다양한 설정을 통해 피처를 생성.
     - 주요 하이퍼파라미터:
       - `entityset`: DFS를 적용할 엔티티셋.
       - `target_dataframe_name`: 피처를 생성할 타겟 엔티티.
       - `max_depth`: 생성할 피처의 복잡성(깊이)를 설정.
       - `agg_primitives`: 집계 프리미티브(예: `sum`, `mean` 등)를 지정.
       - `trans_primitives`: 변환 프리미티브(예: `day`, `month`, `divide` 등)를 지정.

### 1. **기본 제공 프리미티브**

`FeatureTools`는 다양한 기본 제공 프리미티브를 제공합니다. 이들은 변환과 집계 작업에 사용될 수 있습니다.

#### **변환 프리미티브 (Transformation Primitives)**
변환 프리미티브는 데이터의 개별 행에 대해 변환 작업을 수행합니다.

- **Math Operations**:
  - `Absolute`: 절대값을 반환합니다.
  - `AddNumeric`: 두 숫자를 더합니다.
  - `SubtractNumeric`: 두 숫자를 뺍니다.
  - `MultiplyNumeric`: 두 숫자를 곱합니다.
  - `DivideNumeric`: 두 숫자를 나눕니다.

- **Datetime Transformations**:
  - `Year`: 날짜에서 연도를 추출합니다.
  - `Month`: 날짜에서 월을 추출합니다.
  - `Day`: 날짜에서 일을 추출합니다.
  - `Hour`: 시간에서 시각을 추출합니다.
  - `DayOfWeek`: 날짜에서 요일을 추출합니다.
  - `IsWeekend`: 날짜가 주말인지 여부를 반환합니다.

- **Text Transformations**:
  - `Upper`: 텍스트를 모두 대문자로 변환합니다.
  - `Lower`: 텍스트를 모두 소문자로 변환합니다.
  - `Len`: 텍스트의 길이를 반환합니다.

- **Other Transformations**:
  - `IsNull`: 값이 null인지 확인합니다.
  - `Not`: 논리 NOT 연산을 수행합니다.
  - `Percentile`: 값의 백분위를 반환합니다.

#### **집계 프리미티브 (Aggregation Primitives)**
집계 프리미티브는 그룹화된 데이터에서 요약 통계를 생성합니다.

- **Statistical Aggregations**:
  - `Sum`: 합계를 계산합니다.
  - `Mean`: 평균을 계산합니다.
  - `Min`: 최소값을 반환합니다.
  - `Max`: 최대값을 반환합니다.
  - `Std`: 표준 편차를 계산합니다.
  - `Count`: 개수를 계산합니다.
  - `Median`: 중위수를 계산합니다.
  - `Mode`: 최빈값을 반환합니다.

- **Time-based Aggregations**:
  - `TimeSinceLast`: 마지막 발생 이후 경과 시간을 계산합니다.
  - `TimeSinceFirst`: 첫 발생 이후 경과 시간을 계산합니다.

- **Custom Aggregations**:
  - `NumUnique`: 고유 값의 수를 계산합니다.
  - `NumTrue`: 참(`True`) 값의 수를 계산합니다.
  - `Any`: 하나 이상의 참(`True`) 값이 있는지 확인합니다.
  - `All`: 모든 값이 참(`True`)인지 확인합니다.


In [1]:
import featuretools as ft
import pandas as pd

- 엔티티(Entity) - 데이터베이스 RDBMS 구조
- 엔티티셋(Entityset)- 엔티티간의 관계 정의한 집합

--- 
- 자동으로 피처를 생성해준다. 
- DFS ( Deep Feature Synthesis ) 
    - 데이터생성의 핵심기능 
    - DFS 엔티티간의 관계를 활용해서 깊이 있는 피처를 (유의미한) 만든다. 
    
--- 
- 프리미티브 (primitives)
- 피처 생성 함수 ( featuretools에서 어떤 함수로 피처를 만들래 ? 제공하는 함수 )
    - 변환 (Transformer Primitives) - 데이터의 변환 연도, 월, 일 등의 수치 데이터 변환 
    - 집계 (Aggregation Primitives) - 요약 집계, 기초통계치 ( 평균, sum 등등 )

In [5]:
## 데이터 준비 예시 데이터

#고객 테이블
customer_df = pd.DataFrame({
    'customer_id' : [1,2,3],
    'join_date' : ['2024-01-01','2024-02-01','2024-03-01'],
    'region':['north','east','south']
})


orders_df = pd.DataFrame({
    'order_id' : [1,2,3,4,5],
    'customer_id':[1,1,2,2,3],
    'order_date' : ['2024-04-10','2024-04-11','2024-04-15','2024-05-21','2024-07-05'],
    'amount':[10000,20000,30000,40000,45000]
})


In [12]:
## Entityset 생성
es =ft.EntitySet()

## 엔티티 추가

es =es.add_dataframe(dataframe_name = 'customers', dataframe=customer_df, index = 'customer_id')
es =es.add_dataframe(dataframe_name = 'orders', dataframe=orders_df, index = 'order_id')

/opt/anaconda3/envs/bda/lib/python3.8/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(


In [15]:
## 관계 설정 및 추가
relationship = es.add_relationship(
    parent_dataframe_name = 'customers',
    parent_column_name = 'customer_id',
    child_dataframe_name = 'orders',
    child_column_name = 'customer_id'
)

In [16]:
relationship

Entityset: None
  DataFrames:
    customers [Rows: 3, Columns: 3]
    orders [Rows: 5, Columns: 4]
  Relationships:
    orders.customer_id -> customers.customer_id

In [17]:
## 자동 피처 생성
feature_matrix, feature_defs =ft.dfs(entityset= es, target_dataframe_name = 'customers', max_depth=2)

In [20]:
orders_df

,order_id,customer_id,order_date,amount
1,1,1,2024-04-10,10000
2,2,1,2024-04-11,20000
3,3,2,2024-04-15,30000
4,4,2,2024-05-21,40000
5,5,3,2024-07-05,45000


In [18]:
feature_matrix

,COUNT(orders),MAX(orders.amount),MEAN(orders.amount),MIN(orders.amount),SKEW(orders.amount),STD(orders.amount),SUM(orders.amount),DAY(join_date),MONTH(join_date),WEEKDAY(join_date),YEAR(join_date),MODE(orders.DAY(order_date)),MODE(orders.MONTH(order_date)),MODE(orders.WEEKDAY(order_date)),MODE(orders.YEAR(order_date)),NUM_UNIQUE(orders.DAY(order_date)),NUM_UNIQUE(orders.MONTH(order_date)),NUM_UNIQUE(orders.WEEKDAY(order_date)),NUM_UNIQUE(orders.YEAR(order_date))
customer_id,,,,,,,,,,,,,,,,,,,
1,2,20000.0,15000.0,10000.0,NaN,7071.067812,30000.0,1,1,0,2024,10,4,2,2024,2,1,2,1
2,2,40000.0,35000.0,30000.0,NaN,7071.067812,70000.0,1,2,3,2024,15,4,0,2024,2,2,2,1
3,1,45000.0,45000.0,45000.0,NaN,NaN,45000.0,1,3,4,2024,5,7,4,2024,1,1,1,1


In [19]:
feature_defs

[<Feature: COUNT(orders)>,
 <Feature: MAX(orders.amount)>,
 <Feature: MEAN(orders.amount)>,
 <Feature: MIN(orders.amount)>,
 <Feature: SKEW(orders.amount)>,
 <Feature: STD(orders.amount)>,
 <Feature: SUM(orders.amount)>,
 <Feature: DAY(join_date)>,
 <Feature: MONTH(join_date)>,
 <Feature: WEEKDAY(join_date)>,
 <Feature: YEAR(join_date)>,
 <Feature: MODE(orders.DAY(order_date))>,
 <Feature: MODE(orders.MONTH(order_date))>,
 <Feature: MODE(orders.WEEKDAY(order_date))>,
 <Feature: MODE(orders.YEAR(order_date))>,
 <Feature: NUM_UNIQUE(orders.DAY(order_date))>,
 <Feature: NUM_UNIQUE(orders.MONTH(order_date))>,
 <Feature: NUM_UNIQUE(orders.WEEKDAY(order_date))>,
 <Feature: NUM_UNIQUE(orders.YEAR(order_date))>]

In [23]:
## 옵션을 추가 

## 자동 피처 생성
feature_matrix, feature_defs =ft.dfs(
    entityset= es, target_dataframe_name = 'customers',trans_primitives = ['Year', 'Month'] ,max_depth=2
)

In [24]:
feature_matrix

,COUNT(orders),MAX(orders.amount),MEAN(orders.amount),MIN(orders.amount),SKEW(orders.amount),STD(orders.amount),SUM(orders.amount),MONTH(join_date),YEAR(join_date),MODE(orders.MONTH(order_date)),MODE(orders.YEAR(order_date)),NUM_UNIQUE(orders.MONTH(order_date)),NUM_UNIQUE(orders.YEAR(order_date))
customer_id,,,,,,,,,,,,,
1,2,20000.0,15000.0,10000.0,NaN,7071.067812,30000.0,1,2024,4,2024,1,1
2,2,40000.0,35000.0,30000.0,NaN,7071.067812,70000.0,2,2024,4,2024,2,1
3,1,45000.0,45000.0,45000.0,NaN,NaN,45000.0,3,2024,7,2024,1,1


In [26]:
df = pd.read_excel('Online_retail_II.xlsx')

In [27]:
#데이터셋 샘플 10,000개 추출
df=df.loc[:10000]
df=df.dropna()

In [28]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
9672,490147,20983,12 PENCILS TALL TUBE RED SPOTTY,48,2009-12-04 09:31:00,0.85,13408.0,United Kingdom
9673,490147,84879,ASSORTED COLOUR BIRD ORNAMENT,160,2009-12-04 09:31:00,1.45,13408.0,United Kingdom
9674,490147,84988,SET OF 72 PINK HEART PAPER DOILIES,48,2009-12-04 09:31:00,1.45,13408.0,United Kingdom
9675,490147,21490,SET OF THREE 50'S GIFT WRAPS,48,2009-12-04 09:31:00,1.65,13408.0,United Kingdom


## featuretools를 사용해서 집계 피처를 만ㄷ르어 보자!

In [30]:
## 유니크한 인덱스가 지금 없는 상황
df['transaction_id'] =df.index

In [35]:
## Entityset 생성
es =ft.EntitySet()

# 엔티티 추가 
es =es.add_dataframe(dataframe_name='transactions', dataframe=df, index='transaction_id', time_index='InvoiceDate')

# 기존에 있는 df에서 고객 엔티티를 추가하기 위해서 새롭게 es 담는다.
es =es.normalize_dataframe(base_dataframe_name='transactions',new_dataframe_name='customers', index='Customer ID')

/opt/anaconda3/envs/bda/lib/python3.8/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, time_index
  warnings.warn(


In [37]:
## 변환, 집계 프리미티브 추가
feature_matrix, feature_defs = ft.dfs(
    entityset= es
    ,target_dataframe_name = 'customers'
    ,agg_primitives = ["sum",'mean','count','median']
    ,trans_primitives = ['Year', 'Month','day'] 
    ,max_depth=2)

In [49]:
df[df['Customer ID']==16961.0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,transaction_id
9676,C490148,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-1,2009-12-04 09:37:00,2.95,16961.0,United Kingdom,9676


In [50]:
feature_matrix

,COUNT(transactions),MEAN(transactions.Price),MEAN(transactions.Quantity),MEDIAN(transactions.Price),MEDIAN(transactions.Quantity),SUM(transactions.Price),SUM(transactions.Quantity),DAY(first_transactions_time),MONTH(first_transactions_time),YEAR(first_transactions_time)
Customer ID,,,,,,,,,,
13085.0,26,3.425000,14.461538,2.55,12.0,89.05,376.0,1,12,2009
13078.0,19,3.730526,10.157895,3.39,12.0,70.88,193.0,1,12,2009
15362.0,23,3.628261,6.304348,2.95,6.0,83.45,145.0,1,12,2009
18102.0,30,2.635000,255.033333,2.45,60.0,79.05,7651.0,1,12,2009
12682.0,19,3.560000,11.526316,1.65,12.0,67.64,219.0,1,12,2009
...,...,...,...,...,...,...,...,...,...,...
14286.0,76,1.350000,22.197368,0.85,24.0,102.60,1687.0,4,12,2009
13154.0,18,3.341667,8.444444,3.50,6.0,60.15,152.0,4,12,2009
14564.0,39,3.737949,11.256410,2.55,8.0,145.78,439.0,4,12,2009


In [51]:
#customer 집계한 데이터셋을 -> 어떤 유의미한 패턴이 있는 군집으로 만들어서 새로운 피처를 만들 수 있다.
#기존에는 정답이 있는 데이터셋을 -> 전처리해서 성능을 올리는 것 !
#텍스트 데이터 -> 전처리해서 결국 성능올리는 ML기반 
#비지도학습 어떤 정답이 없는 데이터 분석에서는 -> 전처리 하는 과정이 도메인들이 많이 들어가야 할 경우가 있다.
#고객 데이터를 하고 있으니 -> 고객의 주문, 주문금액 등에 대한 피처를 가지고 -> 전처리하여 -> 새로운 피처를 만들고 ->인사이트까지 도출해 볼까?
feature_matrix

,COUNT(transactions),MEAN(transactions.Price),MEAN(transactions.Quantity),MEDIAN(transactions.Price),MEDIAN(transactions.Quantity),SUM(transactions.Price),SUM(transactions.Quantity),DAY(first_transactions_time),MONTH(first_transactions_time),YEAR(first_transactions_time)
Customer ID,,,,,,,,,,
13085.0,26,3.425000,14.461538,2.55,12.0,89.05,376.0,1,12,2009
13078.0,19,3.730526,10.157895,3.39,12.0,70.88,193.0,1,12,2009
15362.0,23,3.628261,6.304348,2.95,6.0,83.45,145.0,1,12,2009
18102.0,30,2.635000,255.033333,2.45,60.0,79.05,7651.0,1,12,2009
12682.0,19,3.560000,11.526316,1.65,12.0,67.64,219.0,1,12,2009
...,...,...,...,...,...,...,...,...,...,...
14286.0,76,1.350000,22.197368,0.85,24.0,102.60,1687.0,4,12,2009
13154.0,18,3.341667,8.444444,3.50,6.0,60.15,152.0,4,12,2009
14564.0,39,3.737949,11.256410,2.55,8.0,145.78,439.0,4,12,2009


In [59]:
from sklearn.feature_selection import VarianceThreshold

## y의 값을 하나를 지정해서 -> 유의미한 피처들이 무엇이 있는지? 찾아보자!
target_variable = 'COUNT(transactions)'

## 피처를 셀렉션
## 상관계수, 분산 기반

corr_matrix =feature_matrix.corr() 
target_corr = corr_matrix[target_variable]

## 임계값 설정해서 피처 선정
selected_by_corr =target_corr[abs(target_corr)>0.1].index

### 분산
selector = VarianceThreshold(threshold=0.1)
selected_by_variacne=feature_matrix.columns[selector.fit(feature_matrix).get_support()]

/var/folders/sz/167qwb595mqgwry5mm8685n00000gn/T/ipykernel_69872/2137307878.py:9: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix =feature_matrix.corr()


In [61]:
target_corr

COUNT(transactions)              1.000000
MEAN(transactions.Price)        -0.105028
MEAN(transactions.Quantity)     -0.061861
MEDIAN(transactions.Price)      -0.134098
MEDIAN(transactions.Quantity)   -0.062675
SUM(transactions.Price)          0.459782
SUM(transactions.Quantity)       0.068213
Name: COUNT(transactions), dtype: float64

In [63]:
selected_by_corr

Index(['COUNT(transactions)', 'MEAN(transactions.Price)',
       'MEDIAN(transactions.Price)', 'SUM(transactions.Price)'],
      dtype='object')

In [62]:
selected_by_variacne

Index(['COUNT(transactions)', 'MEAN(transactions.Price)',
       'MEAN(transactions.Quantity)', 'MEDIAN(transactions.Price)',
       'MEDIAN(transactions.Quantity)', 'SUM(transactions.Price)',
       'SUM(transactions.Quantity)', 'DAY(first_transactions_time)'],
      dtype='object')

In [68]:
## 공통된 피처만 선정하자

selected_feature_by_corr = set(['COUNT(transactions)', 'MEAN(transactions.Price)',
       'MEDIAN(transactions.Price)', 'SUM(transactions.Price)'])

selected_feature_by_variance = set(['COUNT(transactions)', 'MEAN(transactions.Price)',
       'MEAN(transactions.Quantity)', 'MEDIAN(transactions.Price)',
       'MEDIAN(transactions.Quantity)', 'SUM(transactions.Price)',
       'SUM(transactions.Quantity)', 'DAY(first_transactions_time)'])


common_features=list(selected_feature_by_corr.intersection(selected_feature_by_variance))

In [69]:
common_features

['COUNT(transactions)',
 'MEAN(transactions.Price)',
 'SUM(transactions.Price)',
 'MEDIAN(transactions.Price)']

### 우리가 만든 피처로 -> 우리가 피처를 선택하고 -> 해당 피처를 가지고 kmeans 거리 군집을 통해 고객군들의 인사이트가 나올 수 있나?

In [71]:
X = feature_matrix[common_features]

In [74]:
from sklearn.preprocessing import StandardScaler

In [76]:
## K-means

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
## 최적의 K값 찾는 방법은?
## 클러스터의 실루엣, 엘보우, 시각화를 통해 적정한 K를 찾기 위해 진행



In [78]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,transaction_id
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,3
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,4
...,...,...,...,...,...,...,...,...,...
9672,490147,20983,12 PENCILS TALL TUBE RED SPOTTY,48,2009-12-04 09:31:00,0.85,13408.0,United Kingdom,9672
9673,490147,84879,ASSORTED COLOUR BIRD ORNAMENT,160,2009-12-04 09:31:00,1.45,13408.0,United Kingdom,9673
9674,490147,84988,SET OF 72 PINK HEART PAPER DOILIES,48,2009-12-04 09:31:00,1.45,13408.0,United Kingdom,9674
9675,490147,21490,SET OF THREE 50'S GIFT WRAPS,48,2009-12-04 09:31:00,1.65,13408.0,United Kingdom,9675
